In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
np.set_printoptions(suppress=True)
from skimage import data
from skimage import data, color, exposure
from sklearn.neural_network import MLPClassifier
from PIL import Image
from skimage.feature import (match_descriptors, corner_harris,
                             corner_peaks, ORB, plot_matches)
from skimage.color import rgb2gray

labels = []
features = []

# Load the data
def load_data(data_dir):
    """
    From: https://medium.com/@waleedka/traffic-sign-recognition-with-tensorflow-629dffc391a6#.v471kaepx
    """
    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir)
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []

    category = 0
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f)
                      for f in os.listdir(label_dir)
                      if f.endswith(".jpg")]
        
        # adding an early stop for sake of speed
        stop = 0
        for f in file_names:
            img = cv2.imread(f)
            imresize = process_image(img)
            #plt.imshow(imresize)
            images.append(imresize)
            labels.append(category)
            # remove this to use full data set
            if stop > 70:
                break
            stop += 1
            # end early stop
            
        category += 1

    return features, labels

data_dir = "../input/train"
images, labels = load_data(data_dir)


def process_image(img):
    #image = color.rgb2gray(img)
    image = cv2.resize(img, (160, 90))
    #rsize = img.resize((img.size[0]/4,img.size[1]/4)) # Use PIL to resize
    rsizeArr = np.asarray(image) 
    
    orb = cv2.ORB_create(nfeatures=200)
    kp1, des1 = orb.detectAndCompute(rsizeArr,None)

    #print des1
    return des1


from sklearn.datasets import load_digits

digits = load_digits()
X = digits.data
y = digits.target


dataset_size = len(np.array(features))
dataset = np.array(features).reshape(dataset_size,-1)

print (features)
print (dataset)
print (X)
print (dataset).shape
#print dataset.reshape(-1,1)
print (X).shape
#print labels
#print y
#print X[100]
#print dataset[100]#.flatten().reshape(1, -1)

from sklearn.model_selection import train_test_split
from sklearn import svm
import csv

clf = MLPClassifier(solver='lbfgs', alpha=0.0, hidden_layer_sizes=(18,5), random_state=1)
clf.fit(dataset, labels)

def load_test_from_folder(folder):
    probabilities = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            p = clf.predict_proba(process_image(img).flatten().reshape(1, -1))
            
            lst = ((p.tolist()))
            newlist = []
            for i in lst:
                for j in i:
                    j = '%.8f' % j
                    newlist.append([j])
            lst = newlist
            lst.insert(0, [filename])

            import itertools
            chain = itertools.chain(*lst)

            array_again = np.array(list(chain), dtype=object)

            probabilities.append(list(array_again))
    return probabilities
  
probabilities = load_test_from_folder('/modules/cs342/Assignment2/Data/test')
#print probabilities 

with open('zorb.csv', "w") as output:
    print ('writing')
    writer = csv.writer(output, delimiter=',',lineterminator='\n')
    writer.writerow(['image','ALB','BET','DOL','LAG','NoF','OTHER','SHARK','YFT'])
    writer.writerows(probabilities) 
    #for sublist in probabilities:
    #    writer.writerows(sublist) 
